# Automatización para la generación de las fichas técnicas de los indicadores del PGAR de la CAR Cundinamarca para el periodo 2024-2035

Con este código se puede replicar el ejercicio de la generación de las fichas técnicas d elso indicadores. También puede ser usado para replicar el ejercicio con otro conjunto de indicadores.  


```python
#En caso de que se requiera, se deben instalar las librerias que no vienen instalas por defecto cuando se instala Python.

pip install reportlab
pip install PyPDF2
```

In [1]:
#Se cargan las librerias#
#-----------------------#

#Libreria de manejo de los datos#
import pandas as pd 

#Libreria para la creación de documentos PDF personalizados#
from reportlab.lib.pagesizes import letter #COnfigura el tamaño de página.
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image #clases necesarias para crear el documento PDF
from reportlab.lib import colors # para definir colores
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle #para obtener estilos de párrafo predefinidos
from reportlab.lib.units import inch #para especificar medidas en pulgadas
from reportlab.pdfgen import canvas #para generar el PDF y las páginas
from reportlab.platypus import PageTemplate, BaseDocTemplate, Frame #para crear plantillas de página y marcos
import PyPDF2 #para leer archivos PDF y concatenarlos

#Otras librerias
import os #Para determinar la ruta del proyecto
import glob #para trabajar con rutas de archivos y búsqueda de archivos

In [ ]:
#Se configura la carpeta del proyecto: donde estan los inputs y se exportaran las fichas
#os.chdir("Aqui se coloca la ruta de la carpeta del proyecto (utilice '/' en vez de '\')")

## 1. Se preparan los Datos


In [2]:
#Se importa la base de datos y se guarda en un objeto llamado "indicadores"
indicadores = pd.read_excel('Linea base prospectiva PGAR - fichas tecnicas PR.xlsx', 
                            sheet_name='Línea base', header = 0, dtype=str)

indicadores.head()

C:\Users\bdrt1\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,No.,Nombre del Indicador,Dimensión,Reto,Tipo de indicador,Descripción,Unidad de medida,Orientación del indicador,Tipo de acumulación,Línea de base,Metodología de medición,Formula de cálculo,Periodicidad de medición,Fuentes de Información,Serie disponible,Dificultades para la medición,"Territorialización del indicador (por subregiones, municipios, barrios, veredas)",Observaciones,Responsable,Indicador Geográfico calculado (Cuencas)
0,1.1,Porcentaje de puntos de monitoreo del recurso ...,Recurso hídrico,Calidad del agua,Estado,Porcentaje de puntos de monitoreo del recurso ...,Porcentaje,Reducción,Reducción: Busca que se disminuyan los valores...,2021: 78%,Registros elaborados por la CAR dentro del des...,Sumatoria del número de puntos de monitoreo de...,Anual,Boletín del Índice de Calidad del Agua - CAR,"2014/2015, 2018, 2021","Los datos no se encuentran sistematizados, se ...",Sí,NaN,NaN,"Sí, solo:\n- Río Bogotá\n- Río Alto Suárez"
1,2023-02-01 00:00:00,Índice de vulnerabilidad al desabasticimiento ...,Recurso hídrico,Administración de la oferta hídrica,Estado,Índice de Vulnerabilidad al Desabastecimiento ...,Variable categórica con las siguientes categor...,Reducción,"Flujo: Los resultados de un año, no se acumula...","2022: 2,67 Alta",Se utiliza una matriz de asociación entre el Í...,Se creó una matriz de asociación entre el Índi...,Cuatrienal,"Estudios Nacionales del Agua, IDEAM","2014, 2018 y 2022",NaN,Sí,NaN,NaN,No
2,2023-03-01 00:00:00,Porcentaje de población de la jurisdicción que...,Recurso hídrico,Administración de la oferta hídrica,Estado,Porcentaje de población de la jurisdicción que...,Porcentaje,Reducción,Reducción: Busca que se disminuyan los valores...,"2022: 87,2%",Cálculo del porcentaje de población de la juri...,Sumatoria de la población encontrada en territ...,Anual,"Estudios Nacionales del Agua, IDEAM","2018, 2022",NaN,Sí,NaN,NaN,"Si, todas"
3,2023-04-01 00:00:00,Porcentaje de cobertura natural en microcuenca...,Recurso hídrico,Administración de la oferta hídrica,Estado,Porcentaje de cobertura natural en microcuenca...,"Porcentaje para Rio Bogotá, Negro y Suarez",Aumento,"Flujo: Los resultados de un año, no se acumula...","2016: 36,45%",Conteo,Procesamiento cartográfico,Anual,CAR,"2006, 2016",Información disponible únicamente para cuencas...,Sí,NaN,NaN,"Sí, solo:\n- Río Bogotá\n- Río Alto Suárez"
4,1.5,Porcentaje de puntos de monitoreo de calidad d...,Recurso hídrico,Calidad del agua,Estado,Carga contaminante anual vertida en cuencas co...,Porcentaje,Reducción,Reducción: Busca que se disminuyan los valores...,"2021: 19,5%",Conteo,Sumatoria de puntos de monitoreo que no cumple...,Anual,"Boletines anuales del ICA 2013, 2018 y 2021","2013, 2018 y 2021","Los datos no se encuentran sistematizados, se ...",Sí,NaN,NaN,Sí


In [3]:
#Se preparan los datos
indicadores.drop(indicadores.columns[-1], axis=1, inplace= True) #Se elimina la Ultima columna que no nos sirve (En este caso)

indicadores.loc[indicadores['Nombre del Indicador']=='Índice de vulnerabilidad al desabasticimiento hídrico para año seco'
                , 'Nombre del Indicador'] = 'Índice de vulnerabilidad al desabasticimiento hídrico' #Se cambia el nombre de un indicador


In [4]:
#Se configuran algunas Caracteristicas del texto contenido en las fichas
style_column1 = ParagraphStyle(name='CustomStyle', fontName='Helvetica-Bold') #Definimos un estilo de párrafo llamado para la columna 1 con fuente Helvetica en negrilla
style_column2 = ParagraphStyle(name='CustomStyle', fontName='Helvetica', alignment=1) #Definimos un estilo de párrafo llamado para la columna 2 con fuente Helvetica

## 2. Creacion de las fichas

In [5]:
for i in range(indicadores.shape[0]):
    
    #Se crea una función que agrega una imagen en el encbezado de la hojael titulo.
    def agregar_titulo(canvas, doc):
        canvas.saveState()  # Guarda el estado actual del lienzo
        
        # Inserta la imagen como encabezado
        image_path = 'Header_CAR.png'  # La ruta de la imagen para el encabezado
        image_width = 6.3 * inch  # Ancho de la imagen en pulgadas
        image_height = 0.433 * inch  # Alto de la imagen en pulgadas
        x_position = 0.9842519685 * inch # Distancia desde el margen izquierdo en pulgadas
        y_position = letter[1] - 0.395 * inch - image_height  # Distancia desde el margen superior en pulgadas
        
        image = Image(image_path, width=image_width, height=image_height) # Creamos un objeto de imagen utilizando la ruta de la imagen, ancho y alto especificados
        image.drawOn(canvas, x_position, y_position) # Dibujamos la imagen en el lienzo especificando su posición en las coordenadas (x_position, y_position)
        
        #Se inserta el titulo en cada hoja
        canvas.setFont('Helvetica-Bold', 14) #Configura el estilo de la letra y el tamaño del texto de encabezado           
        canvas.drawCentredString(letter[0] / 2, letter[1] - 1.1 * inch, "Plan de Gestión Ambiental Regional (PGAR) 2024-2035") # Coloca texto centrada en el eje horizontal a la mitad de la página 
        canvas.drawCentredString(letter[0] / 2, letter[1] - 1.3 * inch, "Indicadores del Sistema de Seguimiento y Evaluación") # Coloca texto centrada en el eje horizontal a la mitad de la página 

        canvas.restoreState()  # Restaura el estado previo del lienzo
         
    row = indicadores.iloc[i][1:].rename_axis('variable').to_frame().reset_index().fillna('').rename(columns={i: 'valor'}) #Se toma cada la fila (indicador) para generar la ficha
    
    file = f"Ficha_tecnica_LB_{i+1}.pdf" # Se genera la ruta y el nombre del archivo pdf de cada ficha de indicador
    pdf_file = SimpleDocTemplate(file, pagesize=letter) # Creamos el archivo PDF de la ficha donde se creará la tabla con la información del indicador.
    
    ficha = [] #Se crea una lista vcía que contendrá la información de la ficha.
    
    # Se agrega una página de título
    frame = Frame(0, 0, letter[0], letter[1]) # Creamos un marco (frame) que abarca toda la página 
    template = PageTemplate(id='title', frames=[frame], onPage=agregar_titulo) # Definimos una plantilla de página  que utiliza el marco definido y llama a la función 'agregar_titulo' en cada página
    pdf_file.addPageTemplates([template]) # Agregamos la plantilla de página al documento PDF
    ficha.append(Spacer(1, inch+35)) # Agregamos un espacio en blanco en el contenido del documento
    
    # Creamos una lista de listas llamada 'data' para definir los contenidos de la tabla #
    # Agregamos filas con Paragraphs formateados con los estilos 'style_column1' y 'style_column2'
    # Cada fila corresponde a los datos del dataframe "row" que contoiene la información del indicador.
    data = [['PLAN DE GESTIÓN AMBIENTAL REGIONAL (PGAR) 2024-2035 CAR CUNDINAMARCA', ''],
            [Paragraph(row.iloc[0,0], style_column1),  Paragraph(row.iloc[0,1], style_column2)], 
            [Paragraph(row.iloc[1,0], style_column1),  Paragraph(row.iloc[1,1], style_column2)], 
            [Paragraph(row.iloc[2,0], style_column1),  Paragraph(row.iloc[2,1], style_column2)],
            [Paragraph(row.iloc[3,0], style_column1),  Paragraph(row.iloc[3,1], style_column2)],
            [Paragraph(row.iloc[4,0], style_column1),  Paragraph(row.iloc[4,1], style_column2)],
            [Paragraph(row.iloc[5,0], style_column1),  Paragraph(row.iloc[5,1], style_column2)],
            [Paragraph(row.iloc[6,0], style_column1),  Paragraph(row.iloc[6,1], style_column2)],
            [Paragraph(row.iloc[7,0], style_column1),  Paragraph(row.iloc[7,1], style_column2)],
            [Paragraph(row.iloc[8,0], style_column1),  Paragraph('Año ' + row.iloc[8,1], style_column2)],
            [Paragraph(row.iloc[9,0], style_column1),  Paragraph(row.iloc[9,1], style_column2)],
            [Paragraph(row.iloc[10,0], style_column1), Paragraph(row.iloc[10,1], style_column2)],
            [Paragraph(row.iloc[11,0], style_column1), Paragraph(row.iloc[11,1], style_column2)],
            [Paragraph(row.iloc[12,0], style_column1), Paragraph(row.iloc[12,1], style_column2)],
            [Paragraph(row.iloc[13,0], style_column1), Paragraph(row.iloc[13,1], style_column2)],
            [Paragraph(row.iloc[14,0], style_column1), Paragraph(row.iloc[14,1], style_column2)],
            [Paragraph(row.iloc[15,0], style_column1), Paragraph(row.iloc[15,1], style_column2)],
            [Paragraph(row.iloc[16,0], style_column1), Paragraph(row.iloc[16,1], style_column2)],
            [Paragraph(row.iloc[17,0], style_column1), Paragraph(row.iloc[17,1], style_column2)]]
    
    cuadro = Table(data, colWidths=[185, 380]) # Definimos el ancho de las columnas en la tabla
    
    # Definimos el estilo de la tabla utilizando la lista de tuplas 'TableStyle'
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, -1), colors.white),  # Fondo blanco en toda la tabla
        ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),  # Texto negro en toda la tabla
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Líneas de cuadrícula negras con un grosor de 1
        ('SPAN', (0, 0), (-1, 0)),  # Une las dos columnas de la primera fila
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),  # Fondo de la primera fila de color azul claro
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Estilo de letra negrita para la primera fila
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),  # Alineación al centro para la primera fila
        ('FONTNAME', (0, 1), (0, -1), 'Helvetica-Bold', 13),  # Estilo de letra negrita para la primera columna a partir de la segunda fila
        ('FONTNAME', (1, 1), (1, -1), 'Helvetica', 13),  # Estilo de letra para la segunda columna a partir de la segunda fila
        ('ALIGN', (1, 1), (1, -1), 'CENTER'),  # Alineación al centro para la segunda columna a partir de la segunda fila
    ])
    
    
    cuadro.setStyle(style) # Aplicamos el estilo a la tabla 'cuadro'
    ficha.append(cuadro) # Agregamos la tabla 'cuadro' al contenido del documento
    
    pdf_file.build(ficha) # Genera el documento PDF final utilizando la estructura y contenido definidos en 'ficha'

## 3. Unión de los Archivos en un solo archivo

In [6]:
files_fichas = [i.replace('\\', '/')  for i in glob.glob('Ficha_tecnica_*')] # Crea una lista de archivos que coinciden con el patrón 'Ficha_tecnica_*'. Archivos creado con las fichas.

pdf_merger = PyPDF2.PdfMerger() # Creamos un objeto PdfMerger de PyPDF2 para fusionar archivos PDF

# Iteramos a través de la lista de rutas de archivos PDF en 'files_fichas'
for pdf in files_fichas:
    pdf_merger.append(pdf) # Agregamos cada archivo PDF al objeto PdfMerger

pdf_merger.write('Ficha_Indicadores_LB_PGAR.pdf') # Escribimos el PDF fusionado en el archivo 'Ficha_Indicadores_LB_PGAR.pdf'

pdf_merger.close() # Cerramos el objeto PdfMerger después de haber creado el PDF fusionado

## 4. Se eliminan los archivos creados

In [7]:
# Iteramos a través de la lista de rutas de archivos PDF en 'files_fichas'
for f in files_fichas: 
    if os.path.exists(f): # Verificamos si el archivo existe
        os.remove(f) # Si existe, lo eliminamos
    else:
        print('Archivo no existe') # Si el archivo no existe, mostramos un mensaje indicando que no se encontró